## 중요 가이드
- 디렉토리 설명
- 추가 Dependency 폴더 (예: 소스 폴더) 필요시 설정 방법
- Test 폴더
- get_pipeline() 를 통한 파이프라인 실행

```pythona
from pipelines.abalone.pipeline import get_pipeline

pipeline = get_pipeline(
    region=region,
    role=role,
    default_bucket=default_bucket,
    model_package_group_name=model_package_group_name,
    pipeline_name=pipeline_name,
)
pipeline.upsert(role_arn=role)
execution = pipeline.start()
```
- 파라미터 제공 통한 실행

```python
execution = pipeline.start(
    parameters=dict(
        ProcessingInstanceType="ml.c5.xlarge",
        ModelApprovalStatus="Approved",
    )
)
```

### A SageMaker Pipeline

- TransformStep in Pipelines mishandling source_dir attribute (PyTorch/Script Mode)
    - https://issuehint.com/issue/aws/sagemaker-python-sdk/2549


### Getting some constants

We get some constants from the local execution environment.

In [2]:
%load_ext autoreload
%autoreload 2



In [3]:
import boto3
import sagemaker
import os


region = boto3.Session().region_name
role = sagemaker.get_execution_role()
default_bucket = sagemaker.session.Session().default_bucket()

# Change these to reflect your project/business name or if you want to separate ModelPackageGroup/Pipeline from the rest of your team
model_package_group_name = f"ncf-models"
pipeline_name = f"ncf-pipeline-script"

In [4]:
%store -r s3_input_data_uri
%store -r bucket
%store -r project_prefix




In [3]:
from pipelines.ncf.pipeline import get_pipeline

s3_input_data_uri = 's3://sagemaker-us-east-1-057716757052/NCFModel/data'
region = 'us-east-1'
project_prefix = "Serving-NCF"
endpoint_name = "ncf-codepipeline-endpoint"
code_build_project_name = "ncf-serving-sm-pipeline"
pipeline_name = "ncf-sm-serving-pipeline"
model_package_group_name = "NCF-Model-CodePipeline"
inference_image_uri = "763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.8.1-gpu-py3"
role = "arn:aws:iam::057716757052:role/codebuild-gsmoon"
default_bucket = ' sagemaker-us-east-1-057716757052'
base_job_prefix = pipeline_name
training_instance_type="ml.p3.2xlarge"    

In [13]:
from pipelines.ncf.pipeline import get_pipeline


pipeline = get_pipeline(
    s3_input_data_uri = s3_input_data_uri,    
    project_prefix = project_prefix,
    region = region,
    endpoint_name = endpoint_name, # Sagemaker Endpoint Name
    role= role, # SAGEMAKER_PIPELINE_ROLE_ARN 이 넘어옴.
    default_bucket=default_bucket,
    model_package_group_name= model_package_group_name,
    pipeline_name= pipeline_name,
    base_job_prefix= pipeline_name,
    # processing_instance_type= None,
    training_instance_type="ml.p3.2xlarge",    
)

######### get_pipeline() input parameter ###############
### BASE_DIR: /home/ec2-user/SageMaker/Neural-Collaborative-Filtering-On-SageMaker/3_MLOps/5_sm-serving-codepipeline/codecommit/pipelines/ncf
s3_input_data_uri: s3://sagemaker-us-east-1-057716757052/NCFModel/data
project_prefix: Serving-NCF
role: arn:aws:iam::057716757052:role/codebuild-gsmoon
default_bucket:  sagemaker-us-east-1-057716757052
model_package_group_name: NCF-Model-CodePipeline
endpoint_name: ncf-codepipeline-endpoint
pipeline_name: ncf-sm-serving-pipeline
base_job_prefix: ncf-sm-serving-pipeline
training_instance_type: ml.p3.2xlarge
######### Look at subfolder and files #########
./img
    ./img/pipeline-full.png
./origin-sagemaker-pipelines-project.ipynb
./tox.ini
./codebuild-buildspec.yml
./tests
    ./tests/test_pipelines.py
    ./tests/.ipynb_checkpoints
        ./tests/.ipynb_checkpoints/test_pipelines-checkpoint.py
./LICENSE
./pipelines
    ./pipelines/_utils.py
    ./pipelines/.ipynb_checkpoints
        ./pip

### Submit the pipeline to SageMaker and start execution

Let's submit our pipeline definition to the workflow service. The role passed in will be used by the workflow service to create all the jobs defined in the steps.

In [14]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:057716757052:pipeline/ncf-sm-serving-pipeline',
 'ResponseMetadata': {'RequestId': '3f20208a-e9a7-4abb-bd11-c2ea316308ac',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3f20208a-e9a7-4abb-bd11-c2ea316308ac',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '91',
   'date': 'Sat, 19 Nov 2022 06:28:35 GMT'},
  'RetryAttempts': 0}}

We'll start the pipeline, accepting all the default parameters.

Values can also be passed into these pipeline parameters on starting of the pipeline, and will be covered later. 

In [15]:
execution = pipeline.start()

### Pipeline Operations: examining and waiting for pipeline execution

Now we describe execution instance and list the steps in the execution to find out more about the execution.

In [16]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:057716757052:pipeline/ncf-sm-serving-pipeline',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:057716757052:pipeline/ncf-sm-serving-pipeline/execution/bhc0xxagmq7s',
 'PipelineExecutionDisplayName': 'execution-1668839317155',
 'PipelineExecutionStatus': 'Executing',
 'PipelineExperimentConfig': {'ExperimentName': 'ncf-sm-serving-pipeline',
  'TrialName': 'bhc0xxagmq7s'},
 'CreationTime': datetime.datetime(2022, 11, 19, 6, 28, 37, 91000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 11, 19, 6, 28, 37, 91000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': '200544e0-d630-411b-845f-2171d178657d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '200544e0-d630-411b-845f-2171d178657d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '510',
   'date': 'Sat, 19 Nov 2022 06:28:38 GMT'},
  'RetryAttempts': 0}}

We can wait for the execution by invoking `wait()` on the execution:

In [17]:
execution.wait()

We can list the execution steps to check out the status and artifacts:

In [12]:
execution.list_steps()

[{'StepName': 'LambdaDeployStep',
  'StartTime': datetime.datetime(2022, 11, 19, 6, 25, 6, 435000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 11, 19, 6, 25, 8, 989000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'Lambda': {'Arn': 'arn:aws:lambda:us-east-1:057716757052:function:sagemaker-lambda-step-endpoint-deployment',
    'OutputParameters': [{'Name': 'other_key',
      'Value': 'ncf-codepipeline-endpoint'},
     {'Name': 'body', 'Value': '"Created Endpoint!"'},
     {'Name': 'statusCode', 'Value': '200.0'}]}}},
 {'StepName': 'MyModelCreationStep-CreateModel',
  'StartTime': datetime.datetime(2022, 11, 19, 6, 25, 4, 587000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 11, 19, 6, 25, 5, 687000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:us-east-1:057716757052:model/pipelines-j72eqbj2kk3m-mymodelcreationstep--wzs3s0cpbg'}}},
 {'StepName': 'LambdaA

### Parameterized Executions

We can run additional executions of the pipeline specifying different pipeline parameters. The parameters argument is a dictionary whose names are the parameter names, and whose values are the primitive values to use as overrides of the defaults.

Of particular note, based on the performance of the model, we may want to kick off another pipeline execution, but this time on a compute-optimized instance type and set the model approval status automatically be "Approved". This means that the model package version generated by the `RegisterModel` step will automatically be ready for deployment through CI/CD pipelines, such as with SageMaker Projects.

In [ ]:
execution = pipeline.start(
    parameters=dict(
        InputData= s3_input_data_uri,
        ModelApprovalStatus="Approved",
    )
)

In [ ]:
execution.wait()

In [22]:
execution.list_steps()

[{'StepName': 'NCF-Model-Registry',
  'StartTime': datetime.datetime(2022, 7, 4, 14, 18, 35, 201000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 7, 4, 14, 18, 36, 109000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:057716757052:model-package/ncfmodel/6'}}},
 {'StepName': 'NCF-Training',
  'StartTime': datetime.datetime(2022, 7, 4, 14, 7, 43, 83000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 7, 4, 14, 18, 34, 771000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'TrainingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:057716757052:training-job/pipelines-jkld52r1x1s5-ncf-training-zazzcf3wu5'}}}]